In [1]:
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 
import pickle as pickle 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


data_dir = '/content/drive/MyDrive/flow'

categories = ['daisy', 'dandelion' , 'rose', 'sunflower', 'tulip']

data = []

def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)

        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)


            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224,224))
                image = np.array(image)

                data.append([image, label])

            except Exception as e:
                pass

    print(len(data))
    pik = open('flower.pickle','wb')
    pickle.dump(data, pik)
    pik.close()
make_data()
def load_data():
    pick  = open('flower.pickle', 'rb')
    data = pickle.load(pick)

    pick.close()

#np.random(data)

    feature = []
    labels = []

    for img, label in data:
        feature.append(img)
        labels.append(label)

    feature = np.array(feature, dtype = np.float32)
    feature = feature/ 255.

    labels = np.array(labels)

    return [feature, labels]


4308


In [2]:
feature,label = load_data()
categories = ['daisy', 'dandelion' , 'rose', 'sunflower', 'tulip']
x_train, x_test, y_train, y_test = train_test_split(feature,label,test_size=0.1,shuffle=True)
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)) 
x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)
train_dataset=train_dataset.batch(batch_size=16)


In [3]:
cnn = tf.keras.models.Sequential()

In [4]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same',input_shape=[224, 224, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding='same'))


cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding='same'))


cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding='same'))


cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding='same'))


cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu',padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2,padding='same'))

cnn.add(tf.keras.layers.Flatten())


In [5]:
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))

In [6]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.0001)
cnn.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
cnn.fit(x_train,y_train, validation_data = (x_test, y_test), epochs = 10)

Epoch 1/10
  8/122 [>.............................] - ETA: 1:51:10 - loss: 1.6127 - accuracy: 0.2305

In [ ]:
prediction = cnn(x_test[0:9])
plt.figure(figsize=(14,14))

for i in range(9):
   plt.subplot(3,3,i+1)
   plt.imshow(x_test[i])
   plt.xlabel('Predicted:%s\n Actual: %s'%(categories[np.argmax(prediction[i])],
              categories[y_test[i]]))
   
   plt.xticks([])
plt.show()   


In [ ]:
from keras.preprocessing import image
img = image.load_img('/content/drive/MyDrive/flow/daisy/10437754174_22ec990b77_m.jpg', target_size=(224,224,3))




In [ ]:
import tensorflow as tf
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [ ]:
predictions =cnn.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(categories[np.argmax(score)], 100 * np.max(score))
)